<a href="https://colab.research.google.com/github/FranceCosta/Isopeptor_development/blob/main/notebooks/Isopeptide_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

# @markdown # Install dependencies

!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple isopeptor==0.0.71
!pip install py3Dmol
!pip install Bio

import os
import glob
import requests
from pathlib import Path
from Bio.PDB.PDBList import PDBList
from isopeptor.isopeptide import Isopeptide
import py3Dmol

def downloadAF2(uniprot_acc: str, outdir: str, version='v4') -> bool:
    """

        Download AlphaFold models from the AlphaFold database and parse them to
        keep the structure of the desired seqeunce.

        PARAMETERS
        ----------
        auniprot_acc:str,  Uniprot accession code;
        outdir: str, path to directory where to save PDB structure;
        version: str, alphafold database version (default: v4);

        RETURNS
        bool: False if no structure or a wrong structure has been obtained;

    """

    download_link = "https://alphafold.ebi.ac.uk/files/AF-{}-F1-model_{}.pdb"
    output = os.path.join(outdir, f'{uniprot_acc}.pdb')

    try:
        r = requests.get(
            download_link.format(uniprot_acc, version))
        r.raise_for_status()

    except requests.exceptions.HTTPError as e:
        print(f"During download from the AFDB the")
        print(f"following error occurred:\n{e}")
        return 0

    open(output, 'wb').write(r.content)

    return 1

In [12]:
# @markdown #Parameters

# @markdown Insert PDB code, AFDB code or path (example: 1amx, A0A061LY60, your_protein.pdb)
structure_name = "A0A061LY60" #@param {type:"string"}
# @markdown Higher distance values allow a more permessive search
distance = 1.5 #@param {type:"slider", min:0.5, max:5, step:0.5}
# @markdown Fix asa to 0.1 for a quicker computation
fixed_asa = False #@param {type:"boolean"}
if fixed_asa:
    fixed_asa_value = 0.1
# @markdown Evaluate geometric parameters of isopeptide bonds.
# @markdown If set True, geometric data will be included in the output table.
geometry_evaluation = False #@param {type:"boolean"}

In [13]:
# @markdown #Run

os.makedirs("structures", exist_ok=True)
print("Removing pre-existing structures")
files = glob.glob("structures/*")
for f in files:
    os.remove(f)
# Download structure if not a path
if not os.path.isfile(structure_name):
    if len(structure_name) > 4:
        print(f"Downloading AFDB structure {structure_name}...")
        download_status = downloadAF2(structure_name, "./structures")
        assert download_status == True, "Download from AFDB failed"
        new_structure_name = os.path.join("structures", f"{structure_name}.pdb")

    elif len(structure_name) == 4:
        pdbl = PDBList()
        pdb_filename = pdbl.retrieve_pdb_file(structure_name, file_format="pdb",
                                              pdir="./structures", overwrite=True)
        new_structure_name = os.path.join("structures", f"{structure_name}.pdb")
        os.rename(pdb_filename, new_structure_name)

    else:
        print("Structure should be a PDB code, AFDB code or path: none of this found.")
else:
    print("Moving structure to structures folder.")
    os.rename(structure_name, os.path.join("structures", os.path.basename(structure_name)))
    new_structure_name = os.path.join("structures", os.path.basename(structure_name))

structure_path = os.path.dirname(new_structure_name)

# Run isopeptor
print("Running isopeptor...")
if fixed_asa:
    i = Isopeptide(structure_path, distance=distance, fixed_r_asa=fixed_asa_value)
else:
    i = Isopeptide(structure_path, distance=distance)
i.predict()
if geometry_evaluation:
    i.get_geometry()
print("Isopeptor run ok.")

Removing pre-existing structures
Running isopeptor...
Isopeptor run ok.


In [15]:
# @markdown Show results
i.print_tabular()

view=py3Dmol.view()
view.addModel(open(new_structure_name, 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'gray'}})

for bond in sorted(i.isopeptide_bonds, key=lambda x: x.probability):
    structure = bond.struct_file
    chain = bond.chain
    match_residues = (bond.r1_bond, bond.r_cat, bond.r2_bond)
    prob = bond.probability
    if prob > 0.5:
        view.addStyle({'chain':chain,'resi': match_residues},
                {'stick':{'colorscheme':'greenCarbon'}})
    else:
        view.addStyle({'chain':chain,'resi': match_residues},
                {'stick':{'colorscheme':'redCarbon'}})
view.zoomTo()
view.show()

protein_name	probability	chain	r1_bond	r_cat	r2_bond	r1_bond_name	r_cat_name	r2_bond_name	bond_type	rmsd 	r_asa	template             	bond_length	bond_length_zscore	bond_length_allowed	phi    	psi     	omega  	phi_psi_likelihood	phi_psi_allowed	omega_psi_likelihood	omega_psi_allowed	omega_phi_likelihood	omega_phi_allowed
A0A061LY60  	0.85       	A    	566    	603  	668    	LYS         	ASP       	ASN         	CnaA-like	0.369	0.042	2f68_X_176_209_293   	3.144      	22.146            	False              	161.136	-101.384	5.284  	-10.874           	False          	-8.719              	True             	-11.301             	False            
A0A061LY60  	0.822      	A    	719    	756  	821    	LYS         	ASP       	ASN         	CnaA-like	0.401	0.027	2f68_X_176_209_293   	3.107      	21.695            	False              	146.426	-102.271	-1.865 	-9.548            	True           	-8.316              	True             	-10.061             	True             
A0A061LY60  	0.805      	A    	

3Dmol.js failed to load for some reason. Please check your browser console for error messages.